In [1]:
%load_ext autoreload
%autoreload 2

from visualiser.experiment_ui import ExperimentUI

ui = ExperimentUI(True)

ui.get_ui_object()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
